# Final Project: CS 329e Elements of Data Integration
## Option 2: Apply data enrichment at-scale

Enrichments:
1. create country codes for country values orginally from extinct languages table that do not already have a code associated with them in the master languages table. Doing so will allow countries attribute to become obselete and leave only a single country code attribute in our junction table.
2. using population and life expectacy with outside knowledge of country resources to predict life expectacy rates in 2100 and year 3000
3. using population and life expectacy with outside knowledge of country resources to predict population in 2100 and year 3000
4. using the current head of state, gross national product, and independence year to predict the probability of that country instigating or participating in a war within the next 20 years
5. using directors last name to predict nationality
6. generating reviews for a given movie
7. predicting city livability

## Setup

##### Before running this cell, create the remote connection and assign the IAM role `Vertex AI User` to the service account associated with the connection.

In [ ]:
%%bigquery
create or replace model remote_models.gemini_pro
  remote with connection `projects/daniel-daisy-cs329e/locations/us/connections/vertex_connection`
  options (endpoint = 'gemini-pro');

Query is running:   0%|          |

""


## Scenario: Enriching Master Language's country codes and countries attributes where they are null

In [ ]:
%%bigquery
select * except(data_source, load_time)
from country_stg.Master_Language
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,language,country_codes,is_official,avg_percentage,countries,total_num_speakers,degree_of_endangerment
0,Tobada',None,<NA>,NaN,None,12000,Extinct
1,Siriano,None,<NA>,NaN,Brazil,71,Vulnerable
2,Tabasaran,None,<NA>,NaN,None,128391,Vulnerable
3,Quechua of Cuzco,None,<NA>,NaN,None,1115000,Vulnerable
4,Lakon,None,<NA>,NaN,None,700,Vulnerable


Test the generate text function

In [ ]:
%%bigquery
declare prompt_query STRING default "Given a language, and its associated codes, associated country names, and the language's degree of endangerment, construct a list of countries that language is spoken in, and a list of those countries' associated 3-letter country codes.";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("language", language, "country_codes", country_codes, "countries", countries, "degree_of_endangerment", degree_of_endangerment))) as prompt
    from country_stg.Master_Language
    order by language
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,## !Gan!ne\n\nBased on the information provide...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
1,## A'ingae / Cofan Language Distribution\n\nBa...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
2,"## Aasax\n\nBased on the information provided,...","[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
3,## Abaga Language\n\nBased on the information ...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
4,## Abai Sungai\n\nBased on the information pro...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
5,## Abaza Language\n\n### Countries Spoken\n\n*...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
6,## Abkhaz in Georgia\n\nBased on the informati...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
7,None,[],,"Given a language, and its associated codes, as..."
8,## Acatepec Mazatec\n\n### Countries Spoken\n\...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."
9,## Achagua Language\n\nBased on the informatio...,"[{""category"":1,""probability"":1,""probability_sc...",,"Given a language, and its associated codes, as..."


Tweak the prompt and save the output
phases:
1. "Given a language, and its associated codes, associated country names, and the language's degree of endangerment, construct a list of countries that language is spoken in, and a list of those countries' associated 3-letter country codes.";

2. "Given a language, and its associated codes, associated country names, and the language's degree of endangerment, construct a string of countries that language is spoken in, and a list of those countries' associated 3-letter country codes. *Return the output as json, include only the list of codes and country names in the output*";

3. "Given a language, and its associated codes, associated country names, and the language's degree of endangerment, construct a string of countries that language is spoken in, and a list of those countries' associated 3-letter country codes. *Even if some attributes are null, make educated guesses!* Return the output as json, include only the list of codes and country names in the output";

4. "Given a language, and its associated codes, associated country names, and the language's degree of endangerment, construct a string of countries that language is spoken in, and a list of those countries' associated 3-letter country codes. Even if some attributes are null, make educated guesses! *If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to the countries list. If a country exists in the countries attribute, but its associated code is not in the country_codes attributes, make sure to add that code to your country_codes list*. Return the output as json, include only the list of codes and country names in the output";

5. "Given a language, and its associated codes, associated country names, and the language's degree of endangerment, construct a string of countries that language is spoken in, and a list of those countries' associated 3-letter country codes. Even if *degree of endangerment is* null, make educated guesses! If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to *your* countries list. If a country exists in the countries attribute, but its associated code is not in the country_codes attributes, make sure to add that code to your country_codes list. Return the output as json, include only the list of codes and country names in the output";

6. "*Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken along with their associated 3-letter country codes. Even if multiple attributes are missing or incomplete, make educated predictions based on available information.* If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list, *and vice versa*. Return the output as json, include only the list of codes and country names in the output";

7. "Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken along with their associated 3-letter country codes. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list,  and vice versa. *For example, even if degree of endangerment and countries is null, if country-codes has code 'GEO' in it, the countries should have 'Georgia' in it.* Return the output as json, include only the list of codes and country names in the output";

8. "Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken along with their associated 3-letter country codes. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list,  and vice versa. *Assume that if country codes are provided, they accurately reflect the location where that given language is spoken.* Return the output as json, include only the list of codes and country names in the output";

9. "Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken along with their associated 3-letter country codes. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list,  and vice versa. Assume that if country codes are provided, they accurately reflect the location where that given language is spoken. *If unsure which country the three letter code maps to, consider all possible countries and eliminate using information about those countries from the web.* Return the output as json, include only the list of codes and country names in the output";  

10. "Given a language and its associated attributes-- country names, country codes, and degree of endangerment-- and given all available (code, name) pairings, use this data and historical and linguistic data to construct a string of countries where the language is spoken along with their associated 3-letter country codes. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list,  and vice versa. Assume that if country codes are provided, they accurately reflect the location where that given language is spoken. If unsure which country the three letter code maps to, consider all possible countries and eliminate using information about those countries from the data provided or the web. Return the output for each language as json, and include only the list of codes and country names in the output";

11. "Given a language and its associated attributes-- country names, country codes, and degree of endangerment-- and given key:value (code:name) pairings in the dict attribute, use this data and historical and linguistic data from the web to construct a string of countries where the language is spoken along with their associated 3-letter country codes. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. Return the output as json. Include only the list of codes and country names in the output";

12.  "Given a language and its associated attributes, including country names, country codes, and degree of endangerment, construct a string of countries where the language is spoken along with their associated 3-letter country codes. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. Use the dict attribute povided to reference country name and code pairings. Each associated tuple is separated by a colon and each pair is separated by a comma. Return the output as json. Include only the list of codes and country names in the output";

13. "Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken along and a string of those countries associated 3-letter country codes. For example: GEO is the country Georgia; USA is America, etc. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list, and vice versa. Return the output as json, include only the list of codes and country names in the output."

14. **"Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use these attributes and historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken and a string of those countries associated THREE letter country codes. For example: MEX is the country Mexico; USA is America, etc. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list, and vice versa. Return the output as json.Include only the list of codes and country names and the language in the output."**

-- cartesian products causes a prediction to be made for one language many many times (ai predicts row by row)

-- also less efficient time & resource wise

New logic:
- make a "dictionairy" (but really just a very long string) of all code, name tuples from *Country_Location*
- add a new dict column in ML
- update to set all rows to be the dictionairy

Now the ai can reference this while predicting!

In [ ]:
%%bigquery
ALTER TABLE country_stg.Master_Language ADD COLUMN dict STRING;

Query is running:   0%|          |

""


In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT code, name FROM `country_stg.Country_Location` ')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

tmp = ''
for row in rows:
    tmp += row.code + ':' + row.name + ','
print(tmp)

UPDATE_QUERY = (
    f'UPDATE `country_stg.Master_Language` SET dict = "{tmp}" WHERE 1=1')
query_job = client.query(UPDATE_QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

OMN:Oman,QAT:Qatar,IRQ:Iraq,ARE:United Arab Emirates,BHR:Bahrain,GEO:Georgia,AZE:Azerbaijan,ISR:Israel,JOR:Jordan,ARM:Armenia,SAU:Saudi Arabia,LBN:Lebanon,SYR:Syria,YEM:Yemen,TUR:Turkey,PSE:Palestine,CYP:Cyprus,KWT:Kuwait,MNG:Mongolia,CHN:China,KOR:South Korea,MAC:Macao,TWN:Taiwan,PRK:North Korea,JPN:Japan,HKG:Hong Kong,IDN:Indonesia,KHM:Cambodia,THA:Thailand,TMP:East Timor,PHL:Philippines,MMR:Myanmar,LAO:Laos,BRN:Brunei,SGP:Singapore,MYS:Malaysia,VNM:Vietnam,BTN:Bhutan,UZB:Uzbekistan,TKM:Turkmenistan,KGZ:Kyrgyzstan,BGD:Bangladesh,LKA:Sri Lanka,TJK:Tajikistan,MDV:Maldives,IRN:Iran,NPL:Nepal,IND:India,AFG:Afghanistan,PAK:Pakistan,KAZ:Kazakstan,STP:Sao Tome and Principe,CMR:Cameroon,AGO:Angola,GNQ:Equatorial Guinea,TCD:Chad,GAB:Gabon,CAF:Central African Republic,COG:Congo,COD:Congo, The Democratic Republic of the,MOZ:Mozambique,COM:Comoros,ETH:Ethiopia,ZMB:Zambia,MUS:Mauritius,SYC:Seychelles,ERI:Eritrea,UGA:Uganda,MDG:Madagascar,MYT:Mayotte,BDI:Burundi,TZA:Tanzania,DJI:Djibouti,IOT:Briti

In [ ]:
%%bigquery

declare prompt_query STRING default "Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use these attributes and historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken and a string of those countries associated THREE letter country codes. For example: GEO is the country Georgia, MEX is the country Mexico; USA is America, etc. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list, and vice versa. Return the output as json. Include only the list of codes and country names and the language name in the output.";
create or replace table country_stg_ai.country_predictions_raw_10 as
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("language", language, "country_codes", country_codes, "countries", countries, "degree_of_endangerment", degree_of_endangerment))) as prompt
    from country_stg.Master_Language
    order by language
    limit 10
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ml_generate_text_llm_result, prompt
from country_stg_ai.country_predictions_raw_10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,prompt
0,"```json\n{""countries"": null, ""country_codes"": ...",Given a language and its associated attributes...
1,None,Given a language and its associated attributes...
2,"```json\n{""countries"": ""Bolivia"", ""country_cod...",Given a language and its associated attributes...
3,"```json\n{""countries"":""Colombia, Ecuador"",""cou...",Given a language and its associated attributes...
4,"```json\n{""countries"": null, ""country_codes"": ...",Given a language and its associated attributes...
5,None,Given a language and its associated attributes...
6,"```json\n{""countries"": ""Mexico"", ""country_code...",Given a language and its associated attributes...
7,"```json\n{""countries"": null, ""country_codes"": ...",Given a language and its associated attributes...
8,"```json\n{""countries"":""Georgia, Turkey, Russia...",Given a language and its associated attributes...
9,"```json\n{""countries"": null, ""country_codes"": ...",Given a language and its associated attributes...


No charts were generated by quickchart


Format the output to proper json

In [ ]:
%%bigquery
select ml_generate_text_llm_result, trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as formated_result
from country_stg_ai.country_predictions_raw_10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,formated_result
0,"```json\n{""countries"": null, ""country_codes"": ...","{""countries"": null, ""country_codes"": null, ""de..."
1,None,None
2,"```json\n{""countries"": ""Bolivia"", ""country_cod...","{""countries"": ""Bolivia"", ""country_codes"": ""BOL..."
3,"```json\n{""countries"":""Colombia, Ecuador"",""cou...","{""countries"":""Colombia, Ecuador"",""country_code..."
4,"```json\n{""countries"": null, ""country_codes"": ...","{""countries"": null, ""country_codes"": null, ""de..."
5,None,None
6,"```json\n{""countries"": ""Mexico"", ""country_code...","{""countries"": ""Mexico"", ""country_codes"": ""MEX""..."
7,"```json\n{""countries"": null, ""country_codes"": ...","{""countries"": null, ""country_codes"": null, ""de..."
8,"```json\n{""countries"":""Georgia, Turkey, Russia...","{""countries"":""Georgia, Turkey, Russia"",""countr..."
9,"```json\n{""countries"": null, ""country_codes"": ...","{""countries"": null, ""country_codes"": null, ""de..."


In [ ]:
%%bigquery
create or replace table country_stg_ai.country_predictions_formatted_10 as
  select trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as ml_generate_text_llm_result
  from country_stg_ai.country_predictions_raw_10

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.language') as language,
  json_value(ml_generate_text_llm_result, '$.degree_of_endangerment') as degree_of_endangerment,
  json_value(ml_generate_text_llm_result, '$.country_codes') as country_codes,
  json_value(ml_generate_text_llm_result, '$.countries') as countries
from country_stg_ai.country_predictions_formatted_10

Query is running:   0%|          |

Downloading:   0%|          |

,language,degree_of_endangerment,country_codes,countries
0,None,None,None,None
1,None,None,None,None
2,Acatepec Mazatec,None,MEX,Mexico
3,A'ingae / Cofan,None,"COL, ECU","Colombia, Ecuador"
4,Abkhaz,None,"GEO, TUR, RUS","Georgia, Turkey, Russia"
5,Aasax,Extinct,None,None
6,Abaza,Definitely endangered,None,None
7,Achagua,Severely endangered,BOL,Bolivia
8,Abai Sungai,Definitely endangered,None,None
9,!Gan!ne,Extinct,None,None


apply at a larger scale

In [ ]:
%%bigquery

declare prompt_query STRING default "Given a language and its associated attributes, including country names, country codes, and degree of endangerment, use these attributes and historical and linguistic data to enrich the information by constructing a string of countries where the language is spoken and a string of those countries associated THREE letter country codes. For example: GEO is the country Georgia, MEX is the country Mexico; USA is America, etc. Even if multiple attributes are missing or incomplete, make educated predictions based on available information. If a code exists in the country_codes attribute but that country is not listed in countries, make sure to add that country to your countries list, and vice versa. Return the output as json. Include only the list of codes and country names and the language name in the output.";
create or replace table country_stg_ai.country_predictions_raw_full as
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("language", language, "country_codes", country_codes, "countries", countries, "degree_of_endangerment", degree_of_endangerment))) as prompt
    from country_stg.Master_Language
    order by language
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select creation_time, end_time, query
from `region-us`.INFORMATION_SCHEMA.JOBS
where job_id = 'a441f6b3-d6b4-4b33-a8f0-30e044e7b326'

Query is running:   0%|          |

Downloading: |          |

,creation_time,end_time,query


In [ ]:
%%bigquery
create or replace table country_stg_ai.country_predictions_formatted_full as
  select trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as ml_generate_text_llm_result
  from country_stg_ai.country_predictions_raw_full;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update country_stg.Master_Language m
  set m.country_codes = json_value(p.ml_generate_text_llm_result, '$.country_codes')
  from country_stg_ai.country_predictions_formatted_full p
  where m.language = json_value(p.ml_generate_text_llm_result, '$.language') AND m.country_codes is null;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update country_stg.Master_Language m
  set m.countries = json_value(p.ml_generate_text_llm_result, '$.countries')
  from country_stg_ai.country_predictions_formatted_full p
  where m.language = json_value(p.ml_generate_text_llm_result, '$.language') AND m.countries is null;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*) as prediction_count
from country_stg.Master_Language
where countries is not null AND country_codes is not null AND data_source != "BIRD_data.world";

Query is running:   0%|          |

Downloading:   0%|          |

,prediction_count
0,1511


In [ ]:
%%bigquery
select language, country_codes, countries
from country_stg.Master_Language
where countries is not null AND country_codes is not null AND data_source != "BIRD_data.world"
limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,language,country_codes,countries
0,Maya Languages,BLZ,Belize
1,[South]Mande,CIV,Côte d'Ivoire
2,Nuer,SDN,South Sudan
3,Gur,CIV,Côte d'Ivoire
4,Chiu chau,HKG,Hong Kong
5,Ronga,MOZ,Mozambique
6,Karen,MMR,Myanmar
7,Pangasinan,PHL,Philippines
8,Fur,SDN,Sudan
9,Marathi,"IND, MUS","India, Mauritius"


In [ ]:
%%bigquery
update country_stg.Master_Language
  set data_source = 'data.world_ai' where country_codes is not null AND data_source = 'data.world';

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update country_stg.Master_Language
  set data_source = 'BIRD_ai' where countries is not null AND data_source = 'BIRD';

Query is running:   0%|          |

""


## Scenario : Predicting life expectancy rates in the years 2100 & 3000

In [ ]:
%%bigquery
select * except(data_source, load_time)
from country_stg.Social
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,code,population,life_expectancy,local_name
0,CCK,600,NaN,Cocos (Keeling) Islands
1,PCN,50,NaN,Pitcairn
2,ATA,0,NaN,–
3,CXR,2500,NaN,Christmas Island
4,VAT,1000,NaN,Santa Sede/Città del Vaticano


#### Test the generate text function

In [ ]:
%%bigquery
declare prompt_query STRING default "Predict the life expectancy rate in the year 2100 for each country based on the given country code, population, current life expectancy, and outside knowledge of country resources. Return the output as json, include the country code and life expectancy rate for the year 2100 in the output";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("country_code", code, "population", population, "life_expectancy", life_expectancy))) as prompt
    from country_stg.Social
    order by code
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"I am sorry, I cannot fulfill this request. I d...",None,,Predict the life expectancy rate in the year 2...
1,"```json\n{""country_code"":""AFG"",""life_expectanc...",None,,Predict the life expectancy rate in the year 2...
2,## Life Expectancy Predictions for 2100\n\nHer...,None,,Predict the life expectancy rate in the year 2...
3,"I am sorry, I cannot fulfill this request. I d...",None,,Predict the life expectancy rate in the year 2...
4,## Life Expectancy Predictions for 2100\n\nHer...,None,,Predict the life expectancy rate in the year 2...
5,## Life Expectancy Predictions for 2100\n\nHer...,None,,Predict the life expectancy rate in the year 2...
6,"```json\n{""country_code"":""ANT"",""life_expectanc...",None,,Predict the life expectancy rate in the year 2...
7,"I am sorry, I cannot fulfill this request. I d...",None,,Predict the life expectancy rate in the year 2...
8,"```json\n{""country_code"":""ARG"",""life_expectanc...",None,,Predict the life expectancy rate in the year 2...
9,"```json\n{""country_code"":""ARM"",""life_expectanc...",None,,Predict the life expectancy rate in the year 2...


#### Tweak the prompt and save the output

In [ ]:
%%bigquery
declare prompt_query STRING default "Predict the life expectancy rate in the year 2100 and 3000 for each country using extensive historical data, advanced statistical modeling, and consideration of various socio-economic factors. Incorporate the given country code, current population, and current life expectancy into the analysis. Return the output as only a JSON, including the country code, local name, current population, current life expectancy, and the predicted life expectancy rate for the year 2100 and 3000. Include the predicted life expectancies as float numbers. Do not include a disclaimer or explanation as to how you got these results in the output.";
create or replace table country_stg_ai.life_expectancy_predictions_raw_10 as
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("country_code", code, "population", population, "life_expectancy", cast(life_expectancy as float64), "local_name", local_name))) as prompt
    from country_stg.Social
    order by code
    limit 10
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ml_generate_text_llm_result, prompt
from country_stg_ai.life_expectancy_predictions_raw_10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,prompt
0,## Life Expectancy Predictions\n\n```json\n[\n...,Predict the life expectancy rate in the year 2...
1,"```json\n{""country_code"":""AGO"",""local_name"":""A...",Predict the life expectancy rate in the year 2...
2,"```json\n[\n {\n ""country_code"": ""ANT"",\n ...",Predict the life expectancy rate in the year 2...
3,"```json\n{""country_code"":""ABW"",""life_expectanc...",Predict the life expectancy rate in the year 2...
4,## Life Expectancy Predictions\n\n```json\n[\n...,Predict the life expectancy rate in the year 2...
5,"```json\n{""country_code"":""ARM"",""local_name"":""H...",Predict the life expectancy rate in the year 2...
6,"```json\n{""country_code"":""ALB"",""local_name"":""S...",Predict the life expectancy rate in the year 2...
7,"```json\n{""country_code"":""ARG"",""life_expectanc...",Predict the life expectancy rate in the year 2...
8,"```json\n{""country_code"":""AND"",""life_expectanc...",Predict the life expectancy rate in the year 2...
9,"```json\n{""country_code"":""AFG"",""local_name"":""A...",Predict the life expectancy rate in the year 2...


#### Format the output to proper json

In [ ]:
%%bigquery
select ml_generate_text_llm_result, trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as formated_result
from country_stg_ai.life_expectancy_predictions_raw_10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,formated_result
0,## Life Expectancy Predictions\n\n```json\n[\n...,"## Life Expectancy Predictions[ { ""country..."
1,"```json\n{""country_code"":""AGO"",""local_name"":""A...","{""country_code"":""AGO"",""local_name"":""Angola"",""p..."
2,"```json\n[\n {\n ""country_code"": ""ANT"",\n ...","[ { ""country_code"": ""ANT"", ""local_name""..."
3,"```json\n{""country_code"":""ABW"",""life_expectanc...","{""country_code"":""ABW"",""life_expectancy"":78.4,""..."
4,## Life Expectancy Predictions\n\n```json\n[\n...,"## Life Expectancy Predictions[ { ""country..."
5,"```json\n{""country_code"":""ARM"",""local_name"":""H...","{""country_code"":""ARM"",""local_name"":""Hajastan"",..."
6,"```json\n{""country_code"":""ALB"",""local_name"":""S...","{""country_code"":""ALB"",""local_name"":""Shqipëria""..."
7,"```json\n{""country_code"":""ARG"",""life_expectanc...","{""country_code"":""ARG"",""life_expectancy"":75.1,""..."
8,"```json\n{""country_code"":""AND"",""life_expectanc...","{""country_code"":""AND"",""life_expectancy"":83.5,""..."
9,"```json\n{""country_code"":""AFG"",""local_name"":""A...","{""country_code"":""AFG"",""local_name"":""Afganistan..."


In [ ]:
%%bigquery
create or replace table country_stg_ai.life_expectancy_predictions_formatted_10 as
  select trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as ml_generate_text_llm_result
  from country_stg_ai.life_expectancy_predictions_raw_10

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.country_code') as country_code,
  json_value(ml_generate_text_llm_result, '$.local_name') as local_name,
  json_value(ml_generate_text_llm_result, '$.population') as population,
  json_value(ml_generate_text_llm_result, '$.current_life_expectancy') as current_life_expectancy,
  json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_2100') as predicted_life_expectancy_2100,
  json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_3000') as predicted_life_expectancy_3000
from country_stg_ai.life_expectancy_predictions_formatted_10

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,local_name,population,current_life_expectancy,predicted_life_expectancy_2100,predicted_life_expectancy_3000
0,AND,Andorra,78000,None,None,None
1,AGO,Angola,12878000,38.3,72.5,85.2
2,None,None,None,None,None,None
3,ALB,Shqipëria,3401200,71.6,85.4,102.1
4,None,None,None,None,None,None
5,ABW,Aruba,103000,None,85.2,92.1
6,ARG,Argentina,37032000,None,None,None
7,ARM,Hajastan,3520000,66.4,82.5,95.2
8,AFG,Afganistan/Afqanestan,22720000,45.9,72.5,85.2
9,None,None,None,None,None,None


#### Add new columns ```predicted_life_expectancy_2100``` and ```predicted_life_expectancy_3000``` to Social, which are the AI generated predicted life expectancy rates.

In [ ]:
%%bigquery
alter table country_stg.Social add column life_expectancy_2100 float64;
alter table country_stg.Social add column life_expectancy_3000 float64;

Query is running:   0%|          |

""


#### Inspect the output

In [ ]:
%%bigquery
select * except(data_source, load_time)
from country_stg.Social
where life_expectancy is not null;

Query is running:   0%|          |

Downloading:   0%|          |

,code,population,life_expectancy,local_name,population_2100,population_3000,life_expectancy_2100,life_expectancy_3000
0,AGO,12878000,38.3,Angola,10000000,5000000,NaN,NaN
1,PRT,9997600,75.8,Portugal,None,None,NaN,NaN
2,DNK,5330000,76.5,Danmark,None,None,NaN,NaN
3,GLP,456000,77.0,Guadeloupe,None,None,NaN,NaN
4,AND,78000,83.5,Andorra,78000,78000,NaN,NaN
...,...,...,...,...,...,...,...,...
217,MCO,34000,78.8,Monaco,None,None,NaN,NaN
218,AUS,18886000,79.8,Australia,None,None,NaN,NaN
219,SMR,27000,81.1,San Marino,None,None,NaN,NaN
220,JPN,126714000,80.7,Nihon/Nippon,None,None,NaN,NaN


#### Update country_stg.Social table to include the generated life expectancies

```predicted_life_expectancy_2100```

In [ ]:
%%bigquery
update country_stg.Social set life_expectancy_2100 =
  (select cast(json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_2100') as float64)
   from country_stg_ai.life_expectancy_predictions_formatted_10
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


```predicted_life_expectancy_3000```

In [ ]:
%%bigquery
update country_stg.Social set life_expectancy_3000 =
  (select cast(json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_3000') as float64)
   from country_stg_ai.life_expectancy_predictions_formatted_10
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from country_stg.Social
where life_expectancy_2100 is not null or life_expectancy_3000 is not null
order by code;

Query is running:   0%|          |

Downloading:   0%|          |

,code,population,life_expectancy,local_name,data_source,load_time,population_2100,population_3000,life_expectancy_2100,life_expectancy_3000
0,ABW,103000,78.4,Aruba,bird,2024-01-31 22:35:46.344828+00:00,103000,103000,85.2,92.1
1,AFG,22720000,45.9,Afganistan/Afqanestan,bird,2024-01-31 22:35:46.344828+00:00,15000000,10000000,72.5,85.2
2,AGO,12878000,38.3,Angola,bird,2024-01-31 22:35:46.344828+00:00,10000000,5000000,72.5,85.2
3,ALB,3401200,71.6,Shqipëria,bird,2024-01-31 22:35:46.344828+00:00,2800000,2500000,85.4,102.1
4,ARM,3520000,66.4,Hajastan,bird,2024-01-31 22:35:46.344828+00:00,2500000,1500000,82.5,95.2


### Apply to a larger scale

In [ ]:
%%bigquery
declare prompt_query STRING default "Predict the life expectancy rate in the year 2100 and 3000 for each country using extensive historical data, advanced statistical modeling, and consideration of various socio-economic factors. Incorporate the given country code, current population, and current life expectancy into the analysis. Return the output as only a JSON, including the country code, local name, current population, current life expectancy, and the predicted life expectancy rate for the year 2100 and 3000. Do not include a disclaimer or explanation as to how you got these results in the output.";
create or replace table country_stg_ai.life_expectancy_predictions_full as
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("country_code", code, "population", population, "life_expectancy", cast(life_expectancy as float64), "local_name", local_name))) as prompt
    from country_stg.Social
    order by code
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ml_generate_text_llm_result, prompt
from country_stg_ai.life_expectancy_predictions_full

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,prompt
0,"```json\n{""country_code"":""GBR"",""local_name"":""U...",Predict the life expectancy rate in the year 2...
1,"```json\n{""country_code"":""MYT"",""local_name"":""M...",Predict the life expectancy rate in the year 2...
2,"```json\n{""country_code"":""PNG"",""local_name"":""P...",Predict the life expectancy rate in the year 2...
3,"```json\n{""country_code"":""QAT"",""local_name"":""Q...",Predict the life expectancy rate in the year 2...
4,"```json\n{""country_code"":""TCD"",""local_name"":""T...",Predict the life expectancy rate in the year 2...
...,...,...
234,"```json\n{""country_code"":""KGZ"",""local_name"":""K...",Predict the life expectancy rate in the year 2...
235,"```json\n{""country_code"":""LTU"",""local_name"":""L...",Predict the life expectancy rate in the year 2...
236,"```json\n{""country_code"":""BTN"",""local_name"":""D...",Predict the life expectancy rate in the year 2...
237,"```json\n{""country_code"":""POL"",""local_name"":""P...",Predict the life expectancy rate in the year 2...


In [ ]:
%%bigquery
select ml_generate_text_llm_result, trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as formated_result
from country_stg_ai.life_expectancy_predictions_full

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,formated_result
0,"```json\n{""country_code"":""GBR"",""local_name"":""U...","{""country_code"":""GBR"",""local_name"":""United Kin..."
1,"```json\n{""country_code"":""MYT"",""local_name"":""M...","{""country_code"":""MYT"",""local_name"":""Mayotte"",""..."
2,"```json\n{""country_code"":""PNG"",""local_name"":""P...","{""country_code"":""PNG"",""local_name"":""Papua New ..."
3,"```json\n{""country_code"":""QAT"",""local_name"":""Q...","{""country_code"":""QAT"",""local_name"":""Qatar"",""po..."
4,"```json\n{""country_code"":""TCD"",""local_name"":""T...","{""country_code"":""TCD"",""local_name"":""Tchad/Tsha..."
...,...,...
234,"```json\n{""country_code"":""KGZ"",""local_name"":""K...","{""country_code"":""KGZ"",""local_name"":""Kyrgyzstan..."
235,"```json\n{""country_code"":""LTU"",""local_name"":""L...","{""country_code"":""LTU"",""local_name"":""Lietuva"",""..."
236,"```json\n{""country_code"":""BTN"",""local_name"":""D...","{""country_code"":""BTN"",""local_name"":""Druk-Yul"",..."
237,"```json\n{""country_code"":""POL"",""local_name"":""P...","{""country_code"":""POL"",""local_name"":""Polska"",""p..."


In [ ]:
%%bigquery
create or replace table country_stg_ai.life_expectancy_predictions_formatted_full as
  select trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as ml_generate_text_llm_result
  from country_stg_ai.life_expectancy_predictions_full

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.country_code') as country_code,
  json_value(ml_generate_text_llm_result, '$.local_name') as local_name,
  json_value(ml_generate_text_llm_result, '$.population') as population,
  json_value(ml_generate_text_llm_result, '$.current_life_expectancy') as current_life_expectancy,
  json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_2100') as predicted_life_expectancy_2100,
  json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_3000') as predicted_life_expectancy_3000
from country_stg_ai.life_expectancy_predictions_formatted_full

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,local_name,population,current_life_expectancy,predicted_life_expectancy_2100,predicted_life_expectancy_3000
0,TTO,Trinidad and Tobago,1295000,68.0,82.5,95.0
1,TWN,T’ai-wan,22256000,76.4,85.2,92.1
2,ESP,España,39441700,78.8,85.2,92.1
3,None,None,None,None,None,None
4,NZL,New Zealand/Aotearoa,3862000,77.8,85.2,92.7
...,...,...,...,...,...,...
234,CPV,Cabo Verde,428000,68.9,80.5,95.2
235,NIU,Niue,2000,None,85.5,92.5
236,ECU,Ecuador,12646000,71.1,82.5,95.2
237,PER,Perú/Piruw,25662000,70.0,85.0,100.0


In [ ]:
%%bigquery
alter table country_stg.Social add column life_expectancy_2100 float64;
alter table country_stg.Social add column life_expectancy_3000 float64;

Executing query with job ID: e6341a36-9fc5-49ac-822b-008ca84bbe63
Query executing: 0.22s


ERROR:
 400 Query error: Column already exists: predicted_life_expectancy_2100 at [1:43]

Location: US
Job ID: e6341a36-9fc5-49ac-822b-008ca84bbe63



In [ ]:
%%bigquery
update country_stg.Social set life_expectancy_2100 =
  (select cast(json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_2100') as float64)
   from country_stg_ai.life_expectancy_predictions_formatted_full
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update country_stg.Social set life_expectancy_3000 =
  (select cast(json_value(ml_generate_text_llm_result, '$.predicted_life_expectancy_3000') as float64)
   from country_stg_ai.life_expectancy_predictions_formatted_full
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


## Scenario: Predicting population in the years 2100 & 3000

In [ ]:
%%bigquery
alter table country_stg.Social drop column population_2100;
alter table country_stg.Social drop column population_3000;

Query is running:   0%|          |

""


#### Test the generate text output

In [ ]:
%%bigquery
declare prompt_query STRING default "Predict the population in the year 2100 and 3000 for each country using extensive historical data, advanced statistical modeling, and consideration of various socio-economic factors. Incorporate the given country code, current population, and current life expectancy into the analysis. Return the output as only a JSON, including the country code, local name, current population, and the predicted population for the year 2100 and 3000. Do not include a disclaimer, explanation or note as to how you got these results in the output. Name the predicted population columns population_2100 and population_3000";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("country_code", code, "population", CAST(population AS FLOAT64), "life_expectancy", life_expectancy, "local_name", local_name))) as prompt
    from country_stg.Social
    group by code, local_name, population, life_expectancy
    order by code
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

#### Tweak the prompt and save the output

In [ ]:
%%bigquery
declare prompt_query STRING default "Predict the population in the year 2100 and 3000 for the given country using extensive historical data, advanced statistical modeling, and consideration of various socio-economic factors. Incorporate the given country code, current population, and current life expectancy into the analysis. Return the output as only a JSON, including the country code, local name, current population, and the predicted population for the year 2100 and 3000. Do not include a disclaimer, explanation or note as to how you got these results in the output. Name the predicted population columns population_2100 and population_3000";
create or replace table country_stg_ai.population_predictions_raw_10 as
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("country_code", code, "population", CAST(population AS FLOAT64), "life_expectancy", life_expectancy, "local_name", local_name))) as prompt
    from country_stg.Social
    group by code, local_name, population, life_expectancy
    order by code
    limit 10
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ml_generate_text_llm_result, prompt
from country_stg_ai.population_predictions_raw_10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,prompt
0,"```json\n{""country_code"":""AND"",""life_expectanc...",Predict the population in the year 2100 and 30...
1,"```json\n{""country_code"":""AGO"",""local_name"":""A...",Predict the population in the year 2100 and 30...
2,"{""country_code"":""ANT"",""life_expectancy"":74.7,""...",Predict the population in the year 2100 and 30...
3,"```json\n{""country_code"":""ALB"",""local_name"":""S...",Predict the population in the year 2100 and 30...
4,"```json\n{""country_code"":""ARE"",""local_name"":""A...",Predict the population in the year 2100 and 30...
5,"```json\n{""country_code"":""AIA"",""local_name"":""A...",Predict the population in the year 2100 and 30...
6,"```json\n{""country_code"":""ABW"",""local_name"":""A...",Predict the population in the year 2100 and 30...
7,"```json\n{""country_code"":""ARG"",""local_name"":""A...",Predict the population in the year 2100 and 30...
8,"```json\n{""country_code"":""AFG"",""local_name"":""A...",Predict the population in the year 2100 and 30...
9,"```json\n{""country_code"":""ARM"",""local_name"":""H...",Predict the population in the year 2100 and 30...


#### Format the output to proper json

In [ ]:
%%bigquery
select ml_generate_text_llm_result, trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as formated_result
from country_stg_ai.population_predictions_raw_10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,formated_result
0,"```json\n{""country_code"":""AND"",""life_expectanc...","{""country_code"":""AND"",""life_expectancy"":83.5,""..."
1,"```json\n{""country_code"":""AGO"",""local_name"":""A...","{""country_code"":""AGO"",""local_name"":""Angola"",""p..."
2,"{""country_code"":""ANT"",""life_expectancy"":74.7,""...","{""country_code"":""ANT"",""life_expectancy"":74.7,""..."
3,"```json\n{""country_code"":""ALB"",""local_name"":""S...","{""country_code"":""ALB"",""local_name"":""Shqipëria""..."
4,"```json\n{""country_code"":""ARE"",""local_name"":""A...","{""country_code"":""ARE"",""local_name"":""Al-Imarat ..."
5,"```json\n{""country_code"":""AIA"",""local_name"":""A...","{""country_code"":""AIA"",""local_name"":""Anguilla"",..."
6,"```json\n{""country_code"":""ABW"",""local_name"":""A...","{""country_code"":""ABW"",""local_name"":""Aruba"",""po..."
7,"```json\n{""country_code"":""ARG"",""local_name"":""A...","{""country_code"":""ARG"",""local_name"":""Argentina""..."
8,"```json\n{""country_code"":""AFG"",""local_name"":""A...","{""country_code"":""AFG"",""local_name"":""Afganistan..."
9,"```json\n{""country_code"":""ARM"",""local_name"":""H...","{""country_code"":""ARM"",""local_name"":""Hajastan"",..."


In [ ]:
%%bigquery
create or replace table country_stg_ai.population_predictions_formatted_10 as
  select trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as ml_generate_text_llm_result
  from country_stg_ai.population_predictions_raw_10

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.country_code') as country_code,
  json_value(ml_generate_text_llm_result, '$.local_name') as local_name,
  json_value(ml_generate_text_llm_result, '$.population') as population,
  json_value(ml_generate_text_llm_result, '$.population_2100') as population_2100,
  json_value(ml_generate_text_llm_result, '$.population_3000') as population_3000
from country_stg_ai.population_predictions_formatted_10

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,local_name,population,population_2100,population_3000
0,ANT,Nederlandse Antillen,217000.0,180000,150000
1,ABW,Aruba,103000.0,103000,103000
2,ALB,Shqipëria,3401200.0,2500000,1500000
3,AFG,Afganistan/Afqanestan,22720000.0,15000000,10000000
4,ARM,Hajastan,3520000.0,2500000,1500000
5,ARE,Al-Imarat al-´Arabiya al-Muttahida,2441000.0,10000000,20000000
6,AND,Andorra,78000.0,55000,38000
7,AGO,Angola,12878000.0,14500000,16000000
8,AIA,Anguilla,8000.0,5000,3000
9,ARG,Argentina,37032000.0,35000000,25000000


#### Add new columns ```population_2100``` and ```population_3000``` to Social, which are the AI generated predicted populations.

In [ ]:
%%bigquery
alter table country_stg.Social add column population_2100 float64;
alter table country_stg.Social add column population_3000 float64;

Query is running:   0%|          |

""


#### Inspect the output

In [ ]:
%%bigquery
select * except(data_source, load_time)
from country_stg.Social
where population is not null;

Query is running:   0%|          |

Downloading:   0%|          |

,code,population,life_expectancy,local_name,life_expectancy_2100,life_expectancy_3000,population_2100,population_3000
0,STP,147000,65.3,São Tomé e Príncipe,NaN,NaN,NaN,NaN
1,BEL,10239000,77.8,België/Belgique,NaN,NaN,NaN,NaN
2,VAT,1000,NaN,Santa Sede/Città del Vaticano,NaN,NaN,NaN,NaN
3,PCN,50,NaN,Pitcairn,NaN,NaN,NaN,NaN
4,PAN,2856000,75.5,Panamá,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
234,GAB,1226000,50.1,Le Gabon,75.2,105.4,NaN,NaN
235,PNG,4807000,63.1,Papua New Guinea/Papua Niugini,75.2,87.4,NaN,NaN
236,ESH,293000,49.8,As-Sahrawiya,75.2,95.4,NaN,NaN
237,ERI,3850000,55.8,Ertra,75.2,95.4,NaN,NaN


#### Update country_stg.Social table to include the generated population numbers

```population_2100```

In [ ]:
%%bigquery
update country_stg.Social set population_2100 =
  (select cast(json_value(ml_generate_text_llm_result, '$.population_2100') as float64)
   from country_stg_ai.population_predictions_formatted_10
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


```population_3000```

In [ ]:
%%bigquery
update country_stg.Social set population_3000 =
  (select cast(json_value(ml_generate_text_llm_result, '$.population_3000') as float64)
   from country_stg_ai.population_predictions_formatted_10
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from country_stg.Social
where population_2100 is not null or population_3000 is not null
order by code;

Query is running:   0%|          |

Downloading:   0%|          |

,code,population,life_expectancy,local_name,data_source,load_time,life_expectancy_2100,life_expectancy_3000,population_2100,population_3000
0,ABW,103000,78.4,Aruba,bird,2024-01-31 22:35:46.344828+00:00,85.2,92.1,103000.0,103000.0
1,AFG,22720000,45.9,Afganistan/Afqanestan,bird,2024-01-31 22:35:46.344828+00:00,72.5,85.2,15000000.0,10000000.0
2,AGO,12878000,38.3,Angola,bird,2024-01-31 22:35:46.344828+00:00,65.7,85.2,12878000.0,12878000.0
3,AIA,8000,76.1,Anguilla,bird,2024-01-31 22:35:46.344828+00:00,85.2,92.5,6500.0,5000.0
4,ALB,3401200,71.6,Shqipëria,bird,2024-01-31 22:35:46.344828+00:00,85.4,102.1,2800000.0,2500000.0
5,AND,78000,83.5,Andorra,bird,2024-01-31 22:35:46.344828+00:00,92.5,105.0,100000.0,120000.0
6,ANT,217000,74.7,Nederlandse Antillen,bird,2024-01-31 22:35:46.344828+00:00,NaN,NaN,0.0,0.0
7,ARE,2441000,74.1,Al-Imarat al-´Arabiya al-Muttahida,bird,2024-01-31 22:35:46.344828+00:00,85.2,92.5,3456000.0,4567000.0
8,ARG,37032000,75.1,Argentina,bird,2024-01-31 22:35:46.344828+00:00,82.5,95.2,32000000.0,28000000.0
9,ARM,3520000,66.4,Hajastan,bird,2024-01-31 22:35:46.344828+00:00,82.5,105.2,2500000.0,1500000.0


### Apply to a larger scale

In [ ]:
%%bigquery
declare prompt_query STRING default "Predict the population in the year 2100 and 3000 for the given country using extensive historical data, advanced statistical modeling, and consideration of various socio-economic factors. Incorporate the given country code, current population, and current life expectancy into the analysis. Return the output as only a JSON, including the country code, local name, current population, and the predicted population for the year 2100 and 3000. Do not include a disclaimer, explanation or note as to how you got these results in the output. Name the predicted population columns population_2100 and population_3000";
create or replace table country_stg_ai.population_predictions_raw_full as
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("country_code", code, "population", CAST(population AS FLOAT64), "life_expectancy", life_expectancy, "local_name", local_name))) as prompt
    from country_stg.Social
    group by code, local_name, population, life_expectancy
    order by code
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ml_generate_text_llm_result, trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as formated_result
from country_stg_ai.population_predictions_raw_full

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,formated_result
0,"```json\n{""country_code"":""GBR"",""local_name"":""U...","{""country_code"":""GBR"",""local_name"":""United Kin..."
1,"```json\n{""country_code"":""MYT"",""local_name"":""M...","{""country_code"":""MYT"",""local_name"":""Mayotte"",""..."
2,"```json\n{""country_code"":""PNG"",""life_expectanc...","{""country_code"":""PNG"",""life_expectancy"":63.1,""..."
3,"```json\n{""country_code"":""QAT"",""local_name"":""Q...","{""country_code"":""QAT"",""local_name"":""Qatar"",""po..."
4,"```json\n{""country_code"":""TCD"",""local_name"":""T...","{""country_code"":""TCD"",""local_name"":""Tchad/Tsha..."
...,...,...
234,"```json\n{""country_code"":""BOL"",""life_expectanc...","{""country_code"":""BOL"",""life_expectancy"":63.7,""..."
235,"```json\n{""country_code"":""FJI"",""local_name"":""F...","{""country_code"":""FJI"",""local_name"":""Fiji Islan..."
236,"```json\n{""country_code"":""GUY"",""local_name"":""G...","{""country_code"":""GUY"",""local_name"":""Guyana"",""p..."
237,"```json\n{""country_code"":""KGZ"",""local_name"":""K...","{""country_code"":""KGZ"",""local_name"":""Kyrgyzstan..."


In [ ]:
%%bigquery
create or replace table country_stg_ai.population_predictions_formatted_full as
  select trim(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', '')) as ml_generate_text_llm_result
  from country_stg_ai.population_predictions_raw_full

Query is running:   0%|          |

""


#### Update country_stg.Social table to include the generated population numbers

```population_2100```

In [ ]:
%%bigquery
update country_stg.Social set population_2100 =
  (select cast(json_value(ml_generate_text_llm_result, '$.population_2100') as float64)
   from country_stg_ai.population_predictions_formatted_full
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


```population_3000```

In [ ]:
%%bigquery
update country_stg.Social set population_3000 =
  (select cast(json_value(ml_generate_text_llm_result, '$.population_3000') as float64)
   from country_stg_ai.population_predictions_formatted_full
   where code = json_value(ml_generate_text_llm_result, '$.country_code'))
where 1=1

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from country_stg.Social
where population_2100 is not null or population_3000 is not null
order by code;

Query is running:   0%|          |

Downloading:   0%|          |

,code,population,life_expectancy,local_name,data_source,load_time,life_expectancy_2100,life_expectancy_3000,population_2100,population_3000
0,ABW,103000,78.4,Aruba,bird,2024-01-31 22:35:46.344828+00:00,85.2,92.1,105000.0,107000.0
1,AFG,22720000,45.9,Afganistan/Afqanestan,bird,2024-01-31 22:35:46.344828+00:00,72.5,85.2,15000000.0,10000000.0
2,AGO,12878000,38.3,Angola,bird,2024-01-31 22:35:46.344828+00:00,65.7,85.2,14567890.0,16789012.0
3,AIA,8000,76.1,Anguilla,bird,2024-01-31 22:35:46.344828+00:00,85.2,92.5,10000.0,12000.0
4,ALB,3401200,71.6,Shqipëria,bird,2024-01-31 22:35:46.344828+00:00,85.4,102.1,2500000.0,1800000.0
...,...,...,...,...,...,...,...,...,...,...
228,YEM,18112000,59.8,Al-Yaman,bird,2024-01-31 22:35:46.344828+00:00,75.2,85.4,18112000.0,18112000.0
229,YUG,10640000,72.4,Jugoslavija,bird,2024-01-31 22:35:46.344828+00:00,85.2,102.1,10640000.0,10640000.0
230,ZAF,40377000,51.1,South Africa,bird,2024-01-31 22:35:46.344828+00:00,75.2,85.4,32456789.0,25987654.0
231,ZMB,9169000,37.2,Zambia,bird,2024-01-31 22:35:46.344828+00:00,NaN,NaN,12345678.0,98765432.0


## Part 4: Merge changes into target table

### Life Expectancy

```life_expectancy_2100```

In [ ]:
%%bigquery
alter table country_csp.Social add column life_expectancy_2100 float64;

Query is running:   0%|          |

""


```life_expectancy_3000```

In [ ]:
%%bigquery
alter table country_csp.Social add column life_expectancy_3000 float64;

Query is running:   0%|          |

""


### Populations

```population_2100```

In [ ]:
%%bigquery
alter table country_csp.Social add column population_2100 float64;

Query is running:   0%|          |

""


```population_3000```

In [ ]:
%%bigquery
alter table country_csp.Social add column population_3000 float64;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*) as num_records
from country_csp.Social

Query is running:   0%|          |

Downloading:   0%|          |

,num_records
0,239


In [ ]:
%%bigquery
select count(*) as num_updates
from country_csp.Social t join country_stg.Social s
on t.code = s.code
where t.status_flag = true
and (s.life_expectancy_2100 is not null and t.life_expectancy_2100 is null
or s.life_expectancy_3000 is not null and t.life_expectancy_3000 is null
or s.population_2100 is not null and t.population_2100 is null
or s.population_3000 is not null and t.population_3000 is null);

Query is running:   0%|          |

Downloading:   0%|          |

,num_updates
0,238


In [ ]:
%%bigquery
declare current_ts TIMESTAMP;
set current_ts = current_timestamp();

create temp table updates as
  select s.*
  from country_csp.Social t join country_stg.Social s
  on t.code = s.code
  where t.status_flag = true
  and (s.life_expectancy_2100 is not null and t.life_expectancy_2100 is null
  or s.life_expectancy_3000 is not null and t.life_expectancy_3000 is null
  or s.population_2100 is not null and t.population_2100 is null
  or s.population_3000 is not null and t.population_3000 is null);

update country_csp.Social
set discontinue_time = timestamp_sub(current_ts, interval 1 second), status_flag = false
where code in (select code from updates);

insert into country_csp.Social
  (code, local_name, population, life_expectancy, life_expectancy_2100, life_expectancy_3000, population_2100, population_3000, data_source, load_time, effective_time, status_flag)
  (select code, local_name, population, life_expectancy, life_expectancy_2100, life_expectancy_3000, population_2100, population_3000, data_source, load_time, current_ts, true from updates);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select count(*) as num_records
from country_csp.Social

Query is running:   0%|          |

Downloading:   0%|          |

,num_records
0,477


In [ ]:
%%bigquery
update country_stg.Social
  set data_source = 'bird_ai' where
  life_expectancy_2100 is not null
  or life_expectancy_3000 is not null
  or population_2100 is not null
  or population_3000 is not null;

Query is running:   0%|          |

""
